# S3DIS

This notebook lets you instantiate the **[S3DIS](http://buildingparser.stanford.edu/dataset.html)** dataset from scratch and visualize **3D+2D spherical samples**.

Note that you will need **at least 300G** available for the S3DIS raw dataset and **at least 64G** for the processed files at **5cm voxel resolution** and **1024x512 image resolution**.

The S3DIS dataset is composed of **6 Folds**, each representing a distinct building. Images used here are equirectangular panoramic pictures. 

The images and point clouds provided by the [2D-3D-semantics](https://github.com/alexsax/2D-3D-Semantics) repository are not all aligned. The `S3DISFusedDataset` class from `torch_points3d.datasets.segmentation.multimodal.s3dis` deals with correcting some room and camera orientations before processing the data.

This dataset is not too large and as such can be entirely loaded into a **64G RAM** memory. For training, we sample this dataset with **3D spheres and all images seeing at least one point in the spheres**.

In [ ]:
# Select you GPU
I_GPU = 0

In [ ]:
# Uncomment to use autoreload
# %load_ext autoreload
# %autoreload 2

import os
import os.path as osp
import sys
import torch
import numpy as np
from time import time
from omegaconf import OmegaConf
start = time()
import warnings
warnings.filterwarnings('ignore')

torch.cuda.set_device(I_GPU)
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)

from torch_points3d.utils.config import hydra_read
from torch_geometric.data import Data
from torch_points3d.core.multimodal.data import MMData
from torch_points3d.visualization.multimodal_data import visualize_mm_data
from torch_points3d.core.multimodal.image import SameSettingImageData, ImageData
from torch_points3d.datasets.segmentation.multimodal.s3dis import S3DISFusedDataset, OBJECT_LABEL, OBJECT_COLOR
COLORS = OBJECT_COLOR.tolist()
CLASSES = list(OBJECT_LABEL.keys()) + ['unlabelled']

If `visualize_mm_data` does not throw any error but the visualization does not appear, you may need to change your plotly renderer below.

In [ ]:
import plotly.io as pio

pio.renderers.default = 'jupyterlab'        # for local notebook
# pio.renderers.default = 'iframe_connected'  # for remote notebook. Other working (but seemingly slower) options are: 'sphinx_gallery' and 'iframe'

## Dataset creation

The following will instantiate the dataset. If the data is not found at `DATA_ROOT`, the folder structure will be created there and the raw dataset will be downloaded there. 

**Memory-friendly tip** : if you have already downloaded the dataset once and simply want to instantiate a new dataset with different preprocessing (*e.g* change 3D or 2D resolution, mapping parameterization, etc), I recommend you manually replicate the folder hierarchy of your already-existing dataset and create a symlink to its `raw/` directory to avoid downloading and storing (very) large files twice.

You will find the config file ruling the dataset creation at `conf/data/segmentation/multimodal/s3disfused-sparse.yaml`. You may edit this file or create new configs inheriting from this one using Hydra and create the associated dataset by modifying `dataset_config` accordingly in the following cell.

In [ ]:
# Set your dataset root directory, where the data was/will be downloaded
DATA_ROOT = '/path/to/your/dataset/root/directory'
train_is_trainval = False                                     # set to True if you want to the Train set to be Train+Val
sample_per_epoch = 2000                                       # number of spheres sampled in the Train set. Corrects class imbalance. Set to 0 for regularly-sampled spheres
fold = 5                                                      # fold that will be used as Test

dataset_config = 'segmentation/multimodal/s3disfused-sparse'   
models_config = 'segmentation/multimodal/sparseconv3d'       # model family
model_name = 'Res16UNet34-L4-early'                          # specific model

overrides = [
    'task=segmentation',
    f'data={dataset_config}',
    f'models={models_config}',
    f'model_name={model_name}',
    f'data.dataroot={DATA_ROOT}',
    f'data.fold={fold}',
]

cfg = hydra_read(overrides)
# print(OmegaConf.to_yaml(cfg))

The dataset will now be created based on the parsed configuration. I recommend having **at least 300G** available for the S3DIS raw dataset and **at least 64G** for the processed files at **5cm voxel resolution** and **1024x512 image resolution**.

As long as you do not change core dataset parameters, preprocessing should only be performed once for your dataset. It may take some time, **mostly depending on the 3D and 2D resolutions** you choose to work with (the larger the slower). As a rule of thumb, it took me **between 1 and 2 hours** (excluding download) to preprocess the 2D and 3D data the S3DIS dataset.

In [ ]:
# Dataset instantiation
start = time()
dataset = S3DISFusedDataset(cfg.data)
# print(dataset)
print(f"Time = {time() - start:0.1f} sec.")

To visualize the multimodal samples produced by the dataset, we need to remove some of the dataset transforms that affect points, images and mappings. The `sample_real_data` function will be used to get samples without breaking mappings consistency for visualization.

At training and evaluation time, these transforms are used for data augmentation, dynamic size batching (see our [paper](https://arxiv.org/submit/4264152)), etc...

In [ ]:
from torch_geometric.transforms import *
from torch_points3d.core.data_transform import *
from torch_points3d.core.data_transform.multimodal.image import *
from torch_points3d.datasets.base_dataset import BaseDataset
from torch_points3d.datasets.base_dataset_multimodal import BaseDatasetMM

# Transforms on 3D points that we need to exclude for visualization purposes
augmentations_3d = [RandomNoise, RandomRotate, RandomScaleAnisotropic, RandomSymmetry, ShiftVoxels]
exclude_3d_viz = augmentations_3d + [AddFeatsByKeys, Center, GridSampling3D]

# Transforms on 2D images and mappings that we need to exclude for visualization purposes
augmentations_2d = [JitterMappingFeatures, ColorJitter, RandomHorizontalFlip]
exclude_2d_viz = augmentations_2d + [ToFloatImage, Normalize, PickImagesFromMemoryCredit]


def sample_real_data(tg_dataset, idx=0, exclude_3d=None, exclude_2d=None):
    """
    Temporarily remove the 3D and 2D transforms affecting the point 
    positions and images from the dataset to better visualize points 
    and images relative positions.
    """    
    # Remove some 3D transforms
    transform_3d = tg_dataset.transform
    if exclude_3d:
        tg_dataset.transform = BaseDataset.remove_transform(transform_3d, exclude_3d)

    # Remove some 2D transforms, if any
    is_multimodal = hasattr(tg_dataset, 'transform_image')
    if is_multimodal and exclude_2d:
        transform_2d = tg_dataset.transform_image
        tg_dataset.transform_image = BaseDatasetMM.remove_multimodal_transform(transform_2d, exclude_2d)
    
    # Get a sample from the dataset, with transforms excluded
    out = tg_dataset[idx]
    
    # Restore transforms
    tg_dataset.transform = transform_3d
    if is_multimodal and exclude_2d:
        tg_dataset.transform_image = transform_2d
        
    return out

## Visualize a single multimodal sample

We can now pick samples from the train, val and test datasets.

Please refer to `torch_points3d/visualization/multimodal_data` for more details on visualization options.

In [ ]:
# Pick a random sphere in the Train set, with balanced class probabilites
mm_data = sample_real_data(dataset.train_dataset, exclude_3d=exclude_3d_viz, exclude_2d=exclude_2d_viz)

# Pick a random sphere in the Val set
# idx = np.random.randint(len(dataset.val_dataset))
# mm_data = sample_real_data(dataset.val_dataset, idx=idx, exclude_3d=exclude_3d_viz, exclude_2d=exclude_2d_viz)

# Pick a random sphere in the Test set
# idx = np.random.randint(len(dataset.test_dataset[0]))
# mm_data = sample_real_data(dataset.test_dataset[0], idx=idx, exclude_3d=exclude_3d_viz, exclude_2d=exclude_2d_viz)

visualize_mm_data(mm_data, class_names=CLASSES, class_colors=COLORS, figsize=1000, voxel=0.05, show_3d=True, show_2d=True, front='y', alpha=0.3, pointsize=5)

## Visualize a whole S3DIS fold

We can also view a whole fold.

To allow rapid visualization, note the window is subsampled to `voxel=0.25` and the maximum number of points shown is set `max_points=1000000`. Besides `show_2d=False` prevents images from being displayed, which would overload the viewer.

In [ ]:
i_fold = 0  # fold number but does not exactly correspond to the real fold names
mm_data_large = MMData(dataset.train_dataset._datas[i_fold], image=ImageData([dataset.train_dataset._images[i_fold]]))
# mm_data_large = MMData(dataset.val_dataset._datas[i_fold], dataset.val_dataset._images[i_fold], dataset.val_dataset._mappings[i_fold])
# mm_data_large = MMData(dataset.test_dataset[0]._datas[0], dataset.test_dataset[0]._images[0], dataset.test_dataset[0]._mappings[0])
visualize_mm_data(mm_data_large, class_names=CLASSES, class_colors=COLORS, figsize=1000, voxel=0.25, show_2d=False, max_points=1000000)

## Run inference from pretrained weights and visualize predictions
It is possible to visualize the pointwise predictions and errors from a model. 

To do so, we will use the pretrained weights made available with this project. See `README.md` to get the download links and manually place the `.pt` files locally. You will need to provide `checkpoint_dir` where you saved those files in the next cell.

In [ ]:
from torch_points3d.models.model_factory import instantiate_model

# Set your parameters
checkpoint_dir = '/directory/containing/your/checkpoint/file'

# Create the model
print(f"Creating model: {cfg.model_name}")
model = instantiate_model(cfg, dataset)
# print(model)

# Load the checkpoint and recover the 'latest' model weights
checkpoint = torch.load(f'{checkpoint_dir}/{model_name}.pt')
model.load_state_dict_with_same_shape(checkpoint['models']['latest'], strict=False)

# Prepare the model for inference
model = model.eval().cuda()
print('Model loaded')

Now we have loaded the model, we need to run a forward pass on a sample. Howver, if we want to be able to visualize the predictions, we need to pay special attention to which type of 3D and 2D transforms we apply on the data if we do not want to break the mappings. To do so, we will manually apply some sensitive transforms to be able to both infer on the data and visualize it.

In [ ]:
# Pick a random sphere in the Train set, with balanced class probabilites
mm_data = sample_real_data(dataset.train_dataset, exclude_3d=exclude_3d_viz, exclude_2d=exclude_2d_viz)

# Pick a random sphere in the Val set
# idx = np.random.randint(len(dataset.val_dataset))
# mm_data = sample_real_data(dataset.val_dataset, idx=idx, exclude_3d=exclude_3d_viz, exclude_2d=exclude_2d_viz)

# Pick a random sphere in the Test set
# idx = np.random.randint(len(dataset.test_dataset[0]))
# mm_data = sample_real_data(dataset.test_dataset[0], idx=idx, exclude_3d=exclude_3d_viz, exclude_2d=exclude_2d_viz)
# mm_data = sample_real_data(dataset.test_dataset[0], idx=idx, exclude_3d=exclude_3d_viz, exclude_2d=exclude_2d_viz)

# Extract point cloud and images from MMData object
data = mm_data.data.clone()
images = mm_data.modalities['image'].clone()

# For voxel-based 3D backbones such as SparseConv3d and MinkowskiNet, points need to be 
# preprocessed with Center and GridSampling3D. Unfortunately, Center breaks relative 
# positions between points and images. Besides, the combination of Center and GridSampling3D
# may lead to some points being merged into the same voxels, so we must apply it to both the
# inference and visualization data to make sure we have the same voxels. The workaround here 
# is to manually run these while keeping track of the centering offset
center = data.pos.mean(dim=-2, keepdim=True)
data = AddFeatsByKeys(list_add_to_x=[True], feat_names=['pos_z'], delete_feats=[True])(data)          # add z-height to the features
data = Center()(data)                                                                                 # mean-center the data
data = GridSampling3D(data.grid_size, quantize_coords=True, mode='last')(data)                        # quantization for volumetric models

# This last voxelization step with GridSampling3D might have removed some points, so we need
# to update the mappings usign SelectMappingFromPointId. To control the size of the batch, we
# use PickImagesFromMemoryCredit. Besides, 2D models expect normalized float images, which is
# why we call ToFloatImage and Normalize
data, images = SelectMappingFromPointId()(data, images)                                               # update mappings after GridSampling3D
data, images = PickImagesFromMemoryCredit(
    img_size=cfg.data.resolution_2d, 
    k_coverage=cfg.data.multimodal.settings.test_pixel_credit, 
    n_img=cfg.ata.multimodal.settings.k_coverage)(data, images)                                       # select images to respect memory constraints
data, images_infer = ToFloatImage()(data, images.clone())                                             # convert uint8 images to float
data, images_infer = Normalize()(data, images_infer)                                                  # RGB normalization

# Create a MMData for inference
mm_data_infer = MMData(data, image=images_infer)

# Create a MMBatch and run inference
batch = MMBatch.from_mm_data_list([mm_data_infer])
model.set_input(batch, model.device)
model(batch)

# Create a MMData for visualization
data.pos += center
mm_data = MMData(data, image=images)

# Recover the predicted labels for visualization
mm_data.data.pred = model.output.detach().cpu().argmax(1)

In [ ]:
visualize_mm_data(mm_data, figsize=1000, pointsize=3, voxel=0.05, show_2d=True, front='y', class_names=CLASSES, class_colors=COLORS, alpha=0.3, pointsize=5)